In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import json
import nltk
from nltk.corpus import stopwords
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as stop_sp

# Read recipe inputs
trans_json = dataiku.Dataset("Trans_json")
trans_json_df = trans_json.get_dataframe()

stopwords_fr = dataiku.Dataset("stopwords_fr_1")
stopwords_fr_df = stopwords_fr.get_dataframe()
liste_stopword = list(stopwords_fr_df['line'])

# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

token_insatis_df = trans_json_df # For this sample code, simply copy input to output


# Write recipe outputs
#token_insatis = dataiku.Dataset("token_insatis")
#token_insatis.write_with_schema(token_insatis_df)

In [ ]:
import re
trans = token_insatis_df['transcript_json.callData']
liste_dic = []
#new_df = pd.DataFrame(columns = ['NER','score','duree',"nom"])


for i in range(len(trans)):
    dic_inter = {}
    id_conv = trans_json_df['metadata.reconciliation_id'][i]
    dic_inter['nom'] = id_conv
    one_trans = trans[i]
    trans_dict = json.loads(one_trans)
    liste_token_insu = []
    
    for l in trans_dict:
        


        word = l['words']
        for m in word:
            value = m['value']
            matchObj = re.findall('\[.+\]', value)

            if matchObj:
                pass
            
            
            else :
                    
                    
                    score = float(m['score'])
                    if score>= 0 and score < 0.7:
                        
                        value = m['value']
                        
                        
                        liste_token_insu.append(value)
                        
    #print(liste_token_insu)   
    dic_inter['token'] = liste_token_insu
                    

                    

    liste_dic.append(dic_inter)

In [ ]:

stop_nltk = list(stopwords.words('french'))
liste_stop_sp = list(stop_sp)
liste_stop_sp.append("euh")
liste_stop_sp.append("ben")
liste_stop_sp.append("hein")
liste_stop_sp.append("hum")
liste_stop_sp.append("hop")
liste_stop_sp.append("bah")
liste_stop_sp.append("c'est")
liste_stop_sp.append("ouais")
liste_stop_sp.append("j'ai")

liste_stop = set(stop_nltk+liste_stop_sp+liste_stopword)
#print(liste_stop)


In [ ]:
for i in range(len(liste_dic)-1):
    conver = liste_dic[i]
    token_insuff = conver['token']
    if token_insuff == []:
        liste_dic.remove(conver)
#print(len(liste_dic))  

In [ ]:
for conver in liste_dic:
    token_insuff = conver['token']
    #str_token = str(token_insuff)
    #a = str_token.split("'")
    liste_token_net = [word for word in token_insuff if word not in liste_stop]
    conver['token'] = set(liste_token_net)
    '''
    token_insu = conver['token']
    for m in token_insu:
        freq = token_insu.count(m)
        print(m,freq)
    '''

In [ ]:
new_df = pd.DataFrame(liste_dic)
#new_df

In [ ]:
l = []
for conver in liste_dic:
    token_insuff = conver['token']
    for i in token_insuff:
        l.append(i)

In [ ]:
new_df_2 = pd.DataFrame(columns = ["token"])
new_df_2["token"] = l

In [ ]:
token_insatis = dataiku.Dataset("token_insuff_net")
token_insatis.write_with_schema(new_df_2)

In [ ]:
nlp = spacy.load('fr_core_news_md')

In [ ]:

doc = nlp(str(l))
entities, labels, position_start, position_end = [], [], [], []

for ent in doc.ents:
    entities.append(ent)
    labels.append(ent.label_)
    position_start.append(ent.start_char)
    position_end.append(ent.end_char)
    
df = pd.DataFrame({'Entities':entities,'Labels':labels,'Position_Start':position_start, 'Position_End':position_end})

In [ ]:
df